# To create the extracting files for barotropic tide (for 5 days each)

These files will filter and extract the barotropic tide from the ssh_dedac files in "**/bettik/bellemva/MITgcm/MITgcm_filtered_final**"

In [2]:
import numpy as np

In [2]:
date = np.arange(np.datetime64("2012-04-03"),np.datetime64("2012-10-29"),np.timedelta64(5,'D'))

Firstly this cell copys the file extract_tide.py into the number of submitted jobs. 

In [3]:
# deleting the content of file.sh 
file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/file.sh","w")
file.close()


for d in date : 
    file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/file.sh","a")

    command = "cp extract_tide.py extract_tide_" + d.astype('str').replace("-","") + ".py"
    command += "\n"

    file.write(command)

    file.close()


Then, this cell updates the the line 28 to update the variable **init_date** that defines the initial date of the submitted job.

In [5]:
for d in date :

    file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/extract_tide_"+d.astype('str').replace("-","")+".py","r")

    data = file.readlines()

    data[28] = "init_date = np.datetime64(\""+d.astype('str')+"\")\n"
    #data[137] = "    highpass_IT = highpass(lambda_bar,nx,ny,wavnum2D)\n"
    #data[0] = "print(\"job launched\") \n"
    #data[1] = "import os \n"

    file.close()

    file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/extract_tide_"+d.astype('str').replace("-","")+".py","w")

    file.writelines(data)

    file.close()

In [ ]:
data

This cell below creates the submission file with extension .ksh in order to launch the extract_tide.py files 

In [6]:
for d in date : 

    date_str = d.astype('str').replace("-","")

    file = open("/home/bellemva/CCS/submission_files/extract_tide/submission_bar_"+date_str+".sh","w")

    file.write("#!/bin/bash \n")
    file.write("#OAR -n submission_bar_"+date_str+"\n")
    file.write("#OAR -l /nodes=1/core=2,walltime=48:00:00 \n")
    file.write("#OAR --stdout submission_bar_"+date_str+".out \n")
    file.write("#OAR --stderr submission_bar_"+date_str+".err \n")
    file.write("#OAR --project data-ocean \n \n \n")

    file.write("source /home/bellemva/miniconda3/etc/profile.d/conda.sh \n")
    file.write("conda activate pangeo-forge \n")
    file.write("cd /home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/ \n")
    file.write("python extract_tide_"+date_str+".py")

    file.close()

    file = open("/home/bellemva/CCS/submission_files/extract_tide/file.sh","a")

    file.write("oarsub -S /home/bellemva/CCS/submission_files/extract_tide/submission_bar_"+date_str+".sh \n")

    file.close()



### 2 . Joining the hourly MITgcm_bar files into daily ones 

This cell change the time unit of each of the files into "**hours since 2012-01-01 00:00:00**"

In [4]:
dates = np.arange(np.datetime64("2012-04-03"),np.datetime64("2012-10-30"),np.timedelta64(1,"h"))

# deleting the content of file.sh 
file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/file.sh","w")
file.close()

for d in dates:

    command = "ncatted -O -a units,time,o,c,\"hours since 2012-01-01 00:00:00\" "

    date_split = d.astype('str').split("T")

    command += "/bettik/bellemva/MITgcm/MITgcm_bar/MITgcm_bar_"+date_split[0].replace("-","")+"T"+str(int(date_split[1]))+".nc \n"

    file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/file.sh","a")
    
    file.write(command)

    file.close()

    

This cell updates the time coordinate : 

In [2]:
dates = np.arange(np.datetime64("2012-04-03"),np.datetime64("2012-10-30"),np.timedelta64(1,"h"))

# deleting the content of file.sh 
file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/file.sh","w")
file.close()

for d in dates:

    command = "ncap2 -O -s "

    command += "time+="+str((d-np.datetime64("2012-01-01")).astype('int'))

    date_split = d.astype('str').split("T")

    command += " /bettik/bellemva/MITgcm/MITgcm_bar/MITgcm_bar_"+date_split[0].replace("-","")+"T"+str(int(date_split[1]))+".nc "

    command += " /bettik/bellemva/MITgcm/MITgcm_bar/MITgcm_bar_"+date_split[0].replace("-","")+"T"+str(int(date_split[1]))+".nc \n"

    file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/file.sh","a")
    
    file.write(command)

    file.close()

This cell transforms the time coordinate into a record coordinate : 

In [4]:
dates = np.arange(np.datetime64("2012-04-03"),np.datetime64("2012-10-30"),np.timedelta64(1,"h"))

# deleting the content of file.sh 
file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/file.sh","w")
file.close()

for d in dates:

    command = "ncks -O --mk_rec_dmn time "

    date_split = d.astype('str').split("T")

    command += "/bettik/bellemva/MITgcm/MITgcm_bar/MITgcm_bar_"+date_split[0].replace("-","")+"T"+str(int(date_split[1]))+".nc "

    command += "/bettik/bellemva/MITgcm/MITgcm_bar/MITgcm_bar_"+date_split[0].replace("-","")+"T"+str(int(date_split[1]))+".nc \n"

    file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/file.sh","a")
    
    file.write(command)

    file.close()

This cell creates the bash file that will concatenate the hourly barotropic tide datasets into daily ones. Then execute **file.sh**.

In [2]:
dates = np.arange(np.datetime64("2012-04-03"),np.datetime64("2012-10-30"))

# deleting the content of file.sh 
file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/file.sh","w")
file.close()

for d in dates:

    command = "ncrcat "

    for i in range (24) : 

        command += "/bettik/bellemva/MITgcm/MITgcm_bar/MITgcm_bar_"+d.astype('str').replace("-","")+"T"+str(i)+".nc "
    
    command += "/bettik/bellemva/MITgcm/MITgcm_bar/daily_files/MITgcm_bar_"+d.astype('str').replace("-","")+".nc \n"

    
    file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/file.sh","a")
    
    file.write(command)

    file.close()

    

The cell adds the variable $ssh\_bar$ to the **MITgcm_filt** files. It uses the nco command $ncks$ Then execute **file.sh**.

In [3]:
dates = np.arange(np.datetime64("2012-04-05"),np.datetime64("2012-10-30"))

# deleting the content of file.sh 
file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/file.sh","w")
file.close()

for d in dates:

    command = "ncks -A "

    command += "/bettik/bellemva/MITgcm/MITgcm_bar/daily_files/MITgcm_bar_"+d.astype('str').replace("-","")+".nc "

    command += "/bettik/bellemva/MITgcm/MITgcm_filtered_final/MITgcm_filt_"+d.astype('str').replace("-","")+".nc \n"
    
    file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/file.sh","a")
    
    file.write(command)

    file.close()

These cells create the variables **$ssh\_corr=ssh\_dedac-ssh\_bar$** and **$ssh\_igw=ssh\_hf-ssh\_bar$**

In [4]:
dates = np.arange(np.datetime64("2012-04-03"),np.datetime64("2012-10-30"))

# deleting the content of file.sh 
file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/file.sh","w")
file.close()

for d in dates:

    command = "ncap2 -A -s \"ssh_cor=ssh_dedac-ssh_bar\" "

    command += "/bettik/bellemva/MITgcm/MITgcm_filtered_final/MITgcm_filt_"+d.astype('str').replace("-","")+".nc "

    command += "/bettik/bellemva/MITgcm/MITgcm_filtered_final/MITgcm_filt_"+d.astype('str').replace("-","")+".nc \n"
    
    file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/file.sh","a")
    
    file.write(command)

    file.close()


In [5]:
dates = np.arange(np.datetime64("2012-04-03"),np.datetime64("2012-10-30"))

# deleting the content of file.sh 
file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/file.sh","w")
file.close()

for d in dates:

    command = "ncap2 -A -s \"ssh_igw=ssh_hf-ssh_bar\" "

    command += "/bettik/bellemva/MITgcm/MITgcm_filtered_final/MITgcm_filt_"+d.astype('str').replace("-","")+".nc "

    command += "/bettik/bellemva/MITgcm/MITgcm_filtered_final/MITgcm_filt_"+d.astype('str').replace("-","")+".nc \n"
    
    file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/file.sh","a")
    
    file.write(command)

    file.close()

This cell deletes the variables **$ssh\_corr$**, **$ssh\_igw$** and **$ssh\_bar$**.

In [ ]:
dates = np.arange(np.datetime64("2012-04-05"),np.datetime64("2012-10-30"))

# deleting the content of file.sh 
file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/file.sh","w")
file.close()

for d in dates:

    command = "ncks -C -O -x -v ssh_bar,ssh_cor,ssh_igw "

    command += "/bettik/bellemva/MITgcm/MITgcm_filtered_final/MITgcm_filt_"+d.astype('str').replace("-","")+".nc "

    command += "/bettik/bellemva/MITgcm/MITgcm_filtered_final/MITgcm_filt_"+d.astype('str').replace("-","")+".nc \n"
    
    file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/file.sh","a")
    
    file.write(command)

    file.close()


This cell selects the CCS zone and saves it in the **/bettik/bellemva/2022a_CCS_CalVal** repertory. 

In [ ]:
dates = np.arange(np.datetime64("2012-06-01"),np.datetime64("2012-09-01"))

# deleting the content of file.sh 
file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/file.sh","w")
file.close()




### Modifying file name 

In [4]:
missing_date = np.load("./missing_date.npy")

# deleting the content of file.sh 
file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/file.sh","w")
file.close()

for d in missing_date : 

    day_str = d.astype('str').split("T")[0]
    time_index = d.astype('str').split("T")[1]
    time_index_bis = str(int(d.astype('str').split("T")[1]))

    command = "mv /bettik/bellemva/MITgcm/MITgcm_bar/MITgcm_bar_"+day_str.replace("-","")+"T"+time_index+"T"+time_index_bis+".nc "
    command+= "/bettik/bellemva/MITgcm/MITgcm_bar/MITgcm_bar_"+day_str.replace("-","")+"T"+time_index_bis+".nc \n"

    file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/file.sh","a")
    
    file.write(command)

    file.close()

